Jason Cain, Nathan Harms, Marissa Puzan
## Serotonin transport modeling 


Laminar flow model:

## $$ \frac{C_m}{C_o} = \sum_{n} M_n e^{- \beta^2 Gz} $$



from
$$v_z \frac{\partial C}{\partial z} = D \frac{1}{r} \frac{\partial}{\partial r} \left( r \frac{\partial C}{\partial r} \right)$$
where 

$$ Gz = \frac{D L}{V_m R^2} $$

$$ P_w^* = P_w \left( \frac{R}{D} \right) $$

$$ M_n = f( P_w^* ) $$ 

$$ \beta = f( P_w^* ) $$ 

In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from collections import namedtuple

In [2]:
import laminarFlow2

In [3]:
ConditionSet = namedtuple('ConditionSet', ['Concentration', 'Diffusivity', 'Permeability'])

serCondition = ConditionSet(
                Concentration = .1,
                Diffusivity = 6.2424e-8,
                Permeability =7.576e-13)

trypCondition = ConditionSet(
                Concentration = .1,
                Diffusivity = 5.386e-8,
                Permeability = 6.44e-4)

radius = 1./12
length = 25
max_velocity = 5
iterations = 200


Arguments for LaminarFlow(self, n, serConcentration, serGraetz, effSerPermeability, M_Beta_file = 'valuesBetaM.txt'):

In [4]:
Model = laminarFlow2.LaminarFlow(length, radius, serCondition, trypCondition, iterations)

NameError: name 'lengths' is not defined

In [ ]:
plt.plot(Model.serConcentration)

In [ ]:
np.linspace(0,10,20)